<a href="https://colab.research.google.com/github/Diiamon/Election-News-Article-Exploration/blob/main/capstone_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Regex and Imports:

This section includes import statements for various libraries such as **re, requests, PIL, urllib.request, gensim, pandas, numpy, matplotlib, seaborn, wordcloud, nltk, and scipy.stats**. These libraries are used for regular expressions, web connections, image processing, text corpora, data manipulation, visualization, natural language processing, and statistical analysis.

In [ ]:
## Regex
import re

##
import requests # Allows us to make a connection to an external, internet hosted location (eg: Reddit)
from PIL import Image # Allows for image processing
import urllib.request # same as reques
from gensim.corpora import Dictionary # Module that hosts a lof ot English words

## The big 4 + wordclod
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud # create super pretty wordclouds

## All around useful for NLP
import nltk
from nltk.corpus import stopwords # A pre-determined set of stopwords
from nltk.tokenize import word_tokenize # Tokenise our terms!
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from scipy.stats import f_oneway
from scipy.stats import ttest_ind

#Modelling
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [ ]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
# Download some stopwords and punctuation
#Once it is downloaded once, it does not need to be done again

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
lemmatizer = WordNetLemmatizer() #lemmatisation
l_stemmer = LancasterStemmer() # stemmer
p_stemmer = PorterStemmer() #porter stemmer

## Data Cleaning

The code defines functions to clean text data by removing URLs, email addresses, HTML tags, special characters, punctuation, numbers, emojis, and short words. It also includes tokenization, which splits text into individual words or tokens.

In [ ]:
article_table = pd.read_csv('article_table_f')

In [ ]:
article_table

,Article_id,Article,Title,Published_date_id,Source_id
0,52,FOUR independent candidates are standing to fi...,Four candidates for what is thought to be town...,7,6
1,54,First Minister John Swinney has said The SNP w...,SNP to include social tariff on energy and bro...,7,1
2,164,Reform UK insists its plans are “not just anot...,"Do Reform UK’s election claims on tax, immigra...",7,8
3,390,"“If you want politics as pantomime,” Sir Keir ...",Who is Jovan Owusu-Nepaul? Labour’s general el...,7,9
4,242,Shadow health secretary Wes Streeting has urge...,Streeting warns against complacency and giving...,6,4
...,...,...,...,...,...
401,135,Nigel Farage and Reform have made the biggest ...,Farage amasses 39 billion video views as Refor...,21,9
402,83,It’s set to be one of the most dramatic and co...,The Daily T: Your definitive election night guide,21,11
403,308,The Government’s antisemitism adviser has cond...,Tories face backlash over attack on Starmer’s ...,21,4
404,362,"WITH the General Election just days away, cand...",See the key points in the SNP's manifesto ahea...,21,10


### Cleaning Functions

In [ ]:
article_table['cleaned_article'] = article_table['Article'].apply(str.lower)
article_table['cleaned_title'] = article_table['Title'].apply(str.lower)

In [ ]:
def regex_clean(txt, regex):
    """Replace any text matching the regex

    Parameters
    ----------
    txt : string
        A text string that you want to parse and remove matches
    regex : string
        A text string of the regex pattern you want to match

    Returns
    -------
    The same txt string with the matches removes
    """

    return " ".join(re.sub(regex, "", txt).split()) # first split to have it in ['I' ,'am' ,'here'] and the join it with a space between the full texts.

def remove_emoji(string):
    emoji_pattern = re.compile("["
                          "\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "\U0001F300-\U0001F5FF"  # symbols & pictographs
                           "\U0001F600-\U0001F64F"  # emoticons
                           "\U0001F680-\U0001F6FF"  # transport & map symbols
                           "\U0001F700-\U0001F77F"  # alchemical symbols
                           "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           "\U0001FA00-\U0001FA6F"  # Chess Symbols
                           "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           "\U00002702-\U000027B0"  # Dingbats
                           "\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [ ]:
def cleaning_process(df):
  # define a few regex patterns to clean! - This is just a cleaning process (not the only one)
  regex_to_clean = [r'https?://\S+|www\.\S+', # removes web URLs starting with http://, https://, or www..
                    r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b', # removes email addresses.
                    r'<[^>]*>', # removes HTML tags.
                    r'[^a-zA-Z\s]', # removes special characters, punctuation, and numbers, leaving only letters and spaces.
                    r'[@#]\w+', #  removes hashtags and mentions typically found in social media posts.
                    r'\d+', # removes all numeric characters.
                    r'\n|\r|\t', #  removes line breaks and tabs.
                    r'\b\w*\d\w*\b', # removes words that contain digits, which often include product codes or other non-relevant text.
                    r'https?://\S+\?(\S+)', # removes URLs that include query parameters.
                    r'\b\w{1,2}\b', # removes short words with fewer than three characters (optional, as it might remove relevant short words like "an", "is", etc.).
                    r'\[\d+\]|\[citation needed\]', # removes common citation formats like [1], [citation needed].
                    r"[!\"#$%&'()*+,\-./:;<=>?@[\\]^_`{|}~]{2,}", # removes repeated punctuation marks.
                    r'[&%$#@]' # removes artifacts like &, %, etc., which might remain after other cleaning steps.
                  ] #need to add more regex to fully clean

  #step 2.2: Apply the regex clean
  for reg in regex_to_clean:
      df['cleaned_article'] = df['cleaned_article'].apply(regex_clean, regex=reg)
      df['cleaned_title'] = df['cleaned_title'].apply(regex_clean, regex=reg)

  #step 3: apply the emoji removal as well
  df['cleaned_article'] = df['cleaned_article'].apply(remove_emoji)
  df['cleaned_title'] = df['cleaned_title'].apply(remove_emoji)

  return df

In [ ]:
article = cleaning_process(article_table)

In [ ]:
article

,Article_id,Article,Title,Published_date_id,Source_id,cleaned_article,cleaned_title
0,52,FOUR independent candidates are standing to fi...,Four candidates for what is thought to be town...,7,6,four independent candidates are standing fill ...,four candidates for what thought towns first e...
1,54,First Minister John Swinney has said The SNP w...,SNP to include social tariff on energy and bro...,7,1,first minister john swinney has said the snp w...,snp include social tariff energy and broadband...
2,164,Reform UK insists its plans are “not just anot...,"Do Reform UK’s election claims on tax, immigra...",7,8,reform insists its plans are not just another ...,reform uks election claims tax immigration and...
3,390,"“If you want politics as pantomime,” Sir Keir ...",Who is Jovan Owusu-Nepaul? Labour’s general el...,7,9,you want politics pantomime sir keir starmer s...,who jovan owusunepaul labours general election...
4,242,Shadow health secretary Wes Streeting has urge...,Streeting warns against complacency and giving...,6,4,shadow health secretary wes streeting has urge...,streeting warns against complacency and giving...
...,...,...,...,...,...,...,...
401,135,Nigel Farage and Reform have made the biggest ...,Farage amasses 39 billion video views as Refor...,21,9,nigel farage and reform have made the biggest ...,farage amasses billion video views reform domi...
402,83,It’s set to be one of the most dramatic and co...,The Daily T: Your definitive election night guide,21,11,its set one the most dramatic and consequentia...,the daily your definitive election night guide
403,308,The Government’s antisemitism adviser has cond...,Tories face backlash over attack on Starmer’s ...,21,4,the governments antisemitism adviser has conde...,tories face backlash over attack starmers frid...
404,362,"WITH the General Election just days away, cand...",See the key points in the SNP's manifesto ahea...,21,10,with the general election just days away candi...,see the key points the snps manifesto ahead th...


### Tokenisation

In [ ]:
def tokenisation(df):
  df['article_tokens'] = df['cleaned_article'].apply(word_tokenize)
  df['title_tokens'] = df['cleaned_title'].apply(word_tokenize)

  stpwrd = nltk.corpus.stopwords.words('english')
  punc = '!"#$%&()*+, -./:;<=>?@[\]^_`{|}~”“\''
  #generate the list of punctiontion
  punc = [x for x in punc]

  ## extend the list of stopwords to include punctuations as well
  #we'll remove them both anyway
  stpwrd.extend(punc)

  ##apply the cleaning
  df['article_tokens'] = df['article_tokens'].apply(lambda x:[words for words in x if words not in stpwrd])
  df['title_tokens'] = df['title_tokens'].apply(lambda x:[words for words in x if words not in stpwrd])

  ## can get rid of all the terms that have fewer than 2 characters
  df['article_tokens'] = df['article_tokens'].apply(lambda document : [token for token in document if len(token)>2])
  df['title_tokens'] = df['title_tokens'].apply(lambda document : [token for token in document if len(token)>2])

  # Apply stemming and lemmatization
  #df['tokens_stem'] = df['tokens'].apply(lambda tokens: [l_stemmer.stem(word) for word in tokens])
  df['article_tokens_lem'] = df['article_tokens'].apply(lambda tokens: [lemmatizer.lemmatize(word) for word in tokens])
  df['title_tokens_lem'] = df['title_tokens'].apply(lambda tokens: [lemmatizer.lemmatize(word) for word in tokens])
  #df['tokens_stem_2'] = df['tokens'].apply(lambda tokens: [p_stemmer.stem(word) for word in tokens])

  return df

In [ ]:
test = article.head(5).copy()

In [ ]:
t_process = tokenisation(test)

In [ ]:
t_process

,Article_id,Article,Title,Published_date_id,Source_id,cleaned_article,cleaned_title,article_tokens,title_tokens,article_tokens_lem,title_tokens_lem
0,52,FOUR independent candidates are standing to fi...,Four candidates for what is thought to be town...,7,6,four independent candidates are standing fill ...,four candidates for what thought towns first e...,"[four, independent, candidates, standing, fill...","[four, candidates, thought, towns, first, elec...","[four, independent, candidate, standing, fill,...","[four, candidate, thought, town, first, electi..."
1,54,First Minister John Swinney has said The SNP w...,SNP to include social tariff on energy and bro...,7,1,first minister john swinney has said the snp w...,snp include social tariff energy and broadband...,"[first, minister, john, swinney, said, snp, in...","[snp, include, social, tariff, energy, broadba...","[first, minister, john, swinney, said, snp, in...","[snp, include, social, tariff, energy, broadba..."
2,164,Reform UK insists its plans are “not just anot...,"Do Reform UK’s election claims on tax, immigra...",7,8,reform insists its plans are not just another ...,reform uks election claims tax immigration and...,"[reform, insists, plans, another, party, manif...","[reform, uks, election, claims, tax, immigrati...","[reform, insists, plan, another, party, manife...","[reform, uk, election, claim, tax, immigration..."
3,390,"“If you want politics as pantomime,” Sir Keir ...",Who is Jovan Owusu-Nepaul? Labour’s general el...,7,9,you want politics pantomime sir keir starmer s...,who jovan owusunepaul labours general election...,"[want, politics, pantomime, sir, keir, starmer...","[jovan, owusunepaul, labours, general, electio...","[want, politics, pantomime, sir, keir, starmer...","[jovan, owusunepaul, labour, general, election..."
4,242,Shadow health secretary Wes Streeting has urge...,Streeting warns against complacency and giving...,6,4,shadow health secretary wes streeting has urge...,streeting warns against complacency and giving...,"[shadow, health, secretary, wes, streeting, ur...","[streeting, warns, complacency, giving, matche...","[shadow, health, secretary, wes, streeting, ur...","[streeting, warns, complacency, giving, match,..."


## Word Frequency Analysis

This part of the code creates a dictionary of words from the text data and calculates their frequency. It also handles the creation of a DataFrame that lists words along with their frequency in articles and titles.

In [ ]:
t_process.columns

Index(['Article_id', 'Article', 'Title', 'Published_date_id', 'Source_id',
       'cleaned_article', 'cleaned_title', 'article_tokens', 'title_tokens',
       'article_tokens_lem', 'title_tokens_lem'],
      dtype='object')

In [ ]:
def word2(df):
  words = Dictionary(documents = df['tokens_lem'])

  ##
  clean_dictionary = {}

  ##
  for k,v in words.cfs.items():
      if v>1:
          clean_dictionary[words[k]] = v

  return clean_dictionary

In [ ]:
# words = word(t_process)
# words_2 = list(words.keys())

In [ ]:
tokens = tokenisation(article)

In [ ]:
tokens.columns

Index(['Article_id', 'Article', 'Title', 'Published_date_id', 'Source_id',
       'cleaned_article', 'cleaned_title', 'article_tokens', 'title_tokens',
       'article_tokens_lem', 'title_tokens_lem'],
      dtype='object')

### Word Frequency Table

In [ ]:
import pandas as pd
from gensim.corpora import Dictionary
from collections import defaultdict

def word(df):
    # Create dictionaries for article and title tokens
    article_words = Dictionary(documents=df['article_tokens_lem'])
    title_words = Dictionary(documents=df['title_tokens_lem'])

    # Dictionaries to store word frequency and article IDs for both articles and titles
    word_info = defaultdict(lambda: {'article_freq': 0, 'title_freq': 0, 'article_ids': set()})

    # Populate the word_info dictionary for article tokens
    for index, tokens in enumerate(df['article_tokens_lem']):
        article_id = df.loc[index, 'Article_id']
        for word in tokens:
            if article_words.token2id.get(word) is not None:  # Ensure the word is in the dictionary
                word_info[word]['article_freq'] += 1
                word_info[word]['article_ids'].add(article_id)

    # Populate the word_info dictionary for title tokens
    for index, tokens in enumerate(df['title_tokens_lem']):
        for word in tokens:
            if title_words.token2id.get(word) is not None:  # Ensure the word is in the dictionary
                word_info[word]['title_freq'] += 1

    # Create a list for DataFrame construction
    data = []
    for word, info in word_info.items():
        if info['article_freq'] > 0 or info['title_freq'] > 0:  # Only include words with frequency > 0
            data.append([
                word,
                info['article_freq'],
                info['title_freq'],
                list(info['article_ids'])
            ])

    # Convert the data into a DataFrame
    word_df = pd.DataFrame(data, columns=['Word', 'Article_Frequency', 'Title_Frequency', 'Article_IDs'])
    return word_df

# Assuming you have a DataFrame named df
word_table = word(article)


In [ ]:
word_table.head()

,Word,Article_Frequency,Title_Frequency,Article_IDs
0,four,158,2,"[20, 45, 47, 48, 49, 50, 51, 52, 58, 60, 64, 6..."
1,independent,267,2,"[1, 2, 12, 13, 17, 19, 20, 32, 38, 39, 40, 42,..."
2,candidate,975,30,"[0, 1, 2, 3, 5, 6, 7, 8, 9, 11, 12, 13, 16, 19..."
3,standing,211,5,"[1, 2, 3, 5, 7, 8, 9, 11, 13, 16, 21, 22, 28, ..."
4,fill,22,0,"[258, 259, 265, 143, 274, 275, 297, 169, 176, ..."


In [ ]:
word_table['word_id'] = word_table['Word'].astype('category').cat.codes

In [ ]:
word_table.head()

,Word,Article_Frequency,Title_Frequency,Article_IDs,word_id
0,four,158,2,"[20, 45, 47, 48, 49, 50, 51, 52, 58, 60, 64, 6...",8424
1,independent,267,2,"[1, 2, 12, 13, 17, 19, 20, 32, 38, 39, 40, 42,...",10591
2,candidate,975,30,"[0, 1, 2, 3, 5, 6, 7, 8, 9, 11, 12, 13, 16, 19...",3027
3,standing,211,5,"[1, 2, 3, 5, 7, 8, 9, 11, 13, 16, 21, 22, 28, ...",20359
4,fill,22,0,"[258, 259, 265, 143, 274, 275, 297, 169, 176, ...",8053


## Topic Modeling

The code here prepares a corpus for topic modeling using the LDA (Latent Dirichlet Allocation) algorithm. It trains the model to identify topics within the text and associates words with these topics.

In [ ]:
import pandas as pd
from gensim import corpora, models
from collections import defaultdict

# Step 1: Prepare the Corpus
all_tokens_lem = article['article_tokens_lem'].tolist() + article['title_tokens_lem'].tolist()
dictionary = corpora.Dictionary(all_tokens_lem)
corpus = [dictionary.doc2bow(text) for text in all_tokens_lem]

# Step 2: Train the LDA Model
num_topics = 10
lda_model = models.LdaModel(corpus, id2word=dictionary, num_topics=num_topics, passes=15, random_state=42)

In [ ]:
# Step 3: Extract Topic Information
word_topic_info = defaultdict(lambda: {'topics': [], 'topic_freqs': []})
for word_id in dictionary.keys():
    word = dictionary[word_id]
    word_topics = lda_model.get_term_topics(word_id, minimum_probability=0.0)
    for topic_id, freq in word_topics:
        word_topic_info[word]['topics'].append(topic_id)
        word_topic_info[word]['topic_freqs'].append(freq)

# # Create the initial DataFrame with word frequency
# word_table = word(article)
# word_table['word_id'] = word_table['Word'].astype('category').cat.codes

In [ ]:
# Step 4: Integrate with DataFrame
word_table['Topics'] = word_table['Word'].map(lambda w: word_topic_info[w]['topics'] if w in word_topic_info else [])
word_table['Topic_Frequencies'] = word_table['Word'].map(lambda w: word_topic_info[w]['topic_freqs'] if w in word_topic_info else [])

# Display the DataFrame
word_table.head()

,Word,Article_Frequency,Title_Frequency,Article_IDs,word_id,Topics,Topic_Frequencies
0,four,158,2,"[20, 45, 47, 48, 49, 50, 51, 52, 58, 60, 64, 6...",8424,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]","[0.00021399025, 0.0012257358, 0.00070310646, 0..."
1,independent,267,2,"[1, 2, 12, 13, 17, 19, 20, 32, 38, 39, 40, 42,...",10591,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]","[0.0023103475, 0.00015431244, 0.00091897085, 0..."
2,candidate,975,30,"[0, 1, 2, 3, 5, 6, 7, 8, 9, 11, 12, 13, 16, 19...",3027,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]","[0.009531198, 0.00069500046, 0.005038194, 0.00..."
3,standing,211,5,"[1, 2, 3, 5, 7, 8, 9, 11, 13, 16, 21, 22, 28, ...",20359,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]","[0.0011134244, 0.00044080152, 0.0011909573, 0...."
4,fill,22,0,"[258, 259, 265, 143, 274, 275, 297, 169, 176, ...",8053,"[0, 2, 3, 4, 5, 7, 8]","[2.1393156e-05, 3.0331346e-05, 3.7102964e-05, ..."


In [ ]:
word_table.columns

Index(['Word', 'Article_Frequency', 'Title_Frequency', 'Article_IDs',
       'word_id', 'Topics', 'Topic_Frequencies'],
      dtype='object')

### Saving Relevant Tables

In [ ]:
word_table.to_csv('Topic_names_test', index=False)
Article_word = word_table[['word_id', 'Article_IDs']]
Article_word = Article_word.explode('Article_IDs')
Article_word.to_csv('Article_word_f', index=False)

In [ ]:
import pandas as pd

# Assuming `word_table` DataFrame already exists with 'word_id', 'Topics', and 'Topic_Frequencies' columns

# Step 1: Extract Topics and Frequencies
word_topics_data = []

for _, row in word_table.iterrows():
    word_id = row['word_id']
    topics = row['Topics']
    freqs = row['Topic_Frequencies']

    for topic, freq in zip(topics, freqs):
        word_topics_data.append([word_id, topic, freq])

# Step 2: Create the DataFrame
word_topic_df = pd.DataFrame(word_topics_data, columns=['word_id', 'Topic', 'Topic_Frequency'])

# Step 3: (Optional) Merge with word information if necessary
# If you need to include the word itself for better readability or any other details, you can merge this DataFrame with the original `word_table`.
#word_topic_df = word_topic_df.merge(word_table[['word_id', 'Word']], on='word_id', how='left')

# Display the DataFrame
word_topic_df.head()


,word_id,Topic,Topic_Frequency
0,8424,0,0.000214
1,8424,1,0.001226
2,8424,2,0.000703
3,8424,3,0.000152
4,8424,4,0.000270


In [ ]:
word_topic_df.to_csv('word_topic_f', index=False)

In [ ]:
word_table = word_table[['word_id', 'Word', 'Article_Frequency', 'Title_Frequency']]
word_table.to_csv('word_table_f', index=False)

### Possible Topic labelling :

- Topic 0: Political Terms
- Topic 1: Election Process
- Topic 2: Campaign Strategies
- Topic 3: Voter Demographics
- Topic 4: Political Parties
- Topic 5: Election Issues
- Topic 6: Candidate Profiles
- Topic 7: Electoral Regions
- Topic 8: Voting Systems
- Topic 9: Election Outcomes

## Sentiment Analysis

This section includes code for sentiment analysis using rule-based methods like VADER from nltk and machine learning approaches like TextBlob. It also demonstrates how to use pre-trained BERT and RoBERTa models for sentiment analysis.

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch

# Function to calculate sentiment using BERT
def sentiment_bert(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding=True)
    outputs = model(**inputs)
    return outputs.logits.softmax(dim=-1).squeeze().tolist()

# Function to calculate sentiment using RoBERTa
def sentiment_roberta(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding=True)
    outputs = model(**inputs)
    return outputs.logits.softmax(dim=-1).squeeze().tolist()

# Load pre-trained BERT model and tokenizer
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained RoBERTa model and tokenizer
roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base')
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def sentiment(df):

  # Ensure 'tokens_lem' is a string for sentiment analysis
  df['article_tokens_lem_text'] = df['article_tokens_lem'].apply(lambda tokens: ' '.join(tokens) if isinstance(tokens, list) else tokens)
  df['title_tokens_lem_text'] = df['title_tokens_lem'].apply(lambda tokens: ' '.join(tokens) if isinstance(tokens, list) else tokens)

  # Rule-based sentiment analysis with NLTK's Vader
  sid = SentimentIntensityAnalyzer()
  df['article_sentiment_vader'] = df['article_tokens_lem_text'].apply(lambda x: sid.polarity_scores(x)['compound'])
  df['title_sentiment_vader'] = df['title_tokens_lem_text'].apply(lambda x: sid.polarity_scores(x)['compound'])

  # Machine learning sentiment analysis with TextBlob
  df['article_sentiment_textblob'] = df['article_tokens_lem_text'].apply(lambda x: TextBlob(x).sentiment.polarity)
  df['title_sentiment_textblob'] = df['title_tokens_lem_text'].apply(lambda x: TextBlob(x).sentiment.polarity)

  # Combine rule-based and machine learning results
  df['article_sentiment_hybrid'] = df['article_sentiment_vader'] * 0.5 + df['article_sentiment_textblob'] * 0.5
  df['title_sentiment_hybrid'] = df['title_sentiment_vader'] * 0.5 + df['title_sentiment_textblob'] * 0.5

  # BERT sentiment analysis
  df['article_sentiment_bert_-'] = df['article_tokens_lem_text'].apply(lambda x: sentiment_bert(x, bert_model, bert_tokenizer)[0])
  df['article_sentiment_bert_+'] = df['article_tokens_lem_text'].apply(lambda x: sentiment_bert(x, bert_model, bert_tokenizer)[1])

  df['title_sentiment_bert_-'] = df['title_tokens_lem_text'].apply(lambda x: sentiment_bert(x, bert_model, bert_tokenizer)[0])
  df['title_sentiment_bert_+'] = df['title_tokens_lem_text'].apply(lambda x: sentiment_bert(x, bert_model, bert_tokenizer)[1])

  # RoBERTa sentiment analysis
  df['article_sentiment_roberta_+'] = df['article_tokens_lem_text'].apply(lambda x: sentiment_roberta(x, roberta_model, roberta_tokenizer)[0])
  df['article_sentiment_roberta_-'] = df['article_tokens_lem_text'].apply(lambda x: sentiment_roberta(x, roberta_model, roberta_tokenizer)[1])

  df['title_sentiment_roberta_+'] = df['title_tokens_lem_text'].apply(lambda x: sentiment_roberta(x, roberta_model, roberta_tokenizer)[0])
  df['title_sentiment_roberta_-'] = df['title_tokens_lem_text'].apply(lambda x: sentiment_roberta(x, roberta_model, roberta_tokenizer)[1])

  # df results
  df = df[['Article_id',	'Article', 'Title',	'Published_date_id',	'Source_id', 'article_tokens_lem', 'article_tokens_lem_text', 'title_tokens_lem', 'title_tokens_lem_text',
            'article_sentiment_vader',
            'title_sentiment_vader',
           'article_sentiment_textblob',
           'title_sentiment_textblob',
           'article_sentiment_hybrid',
           'title_sentiment_hybrid',
           'article_sentiment_bert_-',
           'article_sentiment_bert_+',
           'title_sentiment_bert_-',
           'title_sentiment_bert_+',
           'article_sentiment_roberta_-',
           'article_sentiment_roberta_+',
           'title_sentiment_roberta_-',
           'title_sentiment_roberta_+'
           ]]

  return df

In [ ]:
# Sample data
data = {
    'Article_id': [1, 2, 3, 4, 5],
    'Title': [
        'Amazing Product Launch',
        'Disappointing Earnings Report',
        'Neutral Outlook on Economy',
        'Exciting Developments in Tech',
        'Concerns Over Market Stability'
    ],
    'Article': [
        'The company has launched an amazing new product that is receiving excellent reviews from customers.',
        'The company’s earnings report was disappointing, falling short of market expectations and resulting in a stock price drop.',
        'Experts have given a neutral outlook on the economy, with no major changes expected in the coming months.',
        'There are exciting developments in the tech industry, with new innovations leading to positive market reactions.',
        'There are growing concerns over market stability due to recent geopolitical tensions and economic uncertainties.'
    ],
    'Sentiment': ['Positive', 'Negative', 'Neutral', 'Positive', 'Negative']
}

# Create DataFrame
df = pd.DataFrame(data)

sid = SentimentIntensityAnalyzer()
df['article_sentiment_bert_-'] = df['Article'].apply(lambda x: sentiment_bert(x, bert_model, bert_tokenizer)[0])
df['article_sentiment_bert_+'] = df['Article'].apply(lambda x: sentiment_bert(x, bert_model, bert_tokenizer)[1])
df['article_sentiment_roberta_-'] = df['Article'].apply(lambda x: sentiment_roberta(x, roberta_model, roberta_tokenizer)[0])
df['article_sentiment_roberta_+'] = df['Article'].apply(lambda x: sentiment_roberta(x, roberta_model, roberta_tokenizer)[1])
df['article_sentiment_vader'] = df['Article'].apply(lambda x: sid.polarity_scores(x)['compound'])
df['article_sentiment_textblob'] = df['Article'].apply(lambda x: TextBlob(x).sentiment.polarity)

df

# From this test:
# BERT = [-, +]
# ROBERTA = [+, -]

,Article_id,Title,Article,Sentiment,article_sentiment_bert_-,article_sentiment_bert_+,article_sentiment_roberta_-,article_sentiment_roberta_+,article_sentiment_vader,article_sentiment_textblob
0,1,Amazing Product Launch,The company has launched an amazing new produc...,Positive,0.327973,0.672028,0.515788,0.484212,0.8402,0.578788
1,2,Disappointing Earnings Report,The company’s earnings report was disappointin...,Negative,0.292081,0.707919,0.518011,0.481989,-0.7096,-0.300000
2,3,Neutral Outlook on Economy,Experts have given a neutral outlook on the ec...,Neutral,0.328459,0.671541,0.515940,0.484060,-0.2960,-0.065625
3,4,Exciting Developments in Tech,There are exciting developments in the tech in...,Positive,0.313984,0.686016,0.516237,0.483763,0.7783,0.221212
4,5,Concerns Over Market Stability,There are growing concerns over market stabili...,Negative,0.279527,0.720473,0.518775,0.481225,-0.5267,0.025000


In [ ]:
#tokens = tokenisation(article)
article_table_s = sentiment(tokens)
article_table_s.to_csv('article_table_s_first', index=False)

In [ ]:
article_table_s = pd.read_csv('article_table_s_first')

In [ ]:
article_table_s.head()

,Article_id,Article,Title,Published_date_id,Source_id,article_tokens_lem,article_tokens_lem_text,title_tokens_lem,title_tokens_lem_text,article_sentiment_vader,...,article_sentiment_hybrid,title_sentiment_hybrid,article_sentiment_bert_-,article_sentiment_bert_+,title_sentiment_bert_-,title_sentiment_bert_+,article_sentiment_roberta_-,article_sentiment_roberta_+,title_sentiment_roberta_-,title_sentiment_roberta_+
0,52,FOUR independent candidates are standing to fi...,Four candidates for what is thought to be town...,7,6,"['four', 'independent', 'candidate', 'standing...",four independent candidate standing fill two v...,"['four', 'candidate', 'thought', 'town', 'firs...",four candidate thought town first election year,0.8399,...,0.541379,0.125000,0.323400,0.676600,0.291945,0.708055,0.485808,0.514192,0.482288,0.517712
1,54,First Minister John Swinney has said The SNP w...,SNP to include social tariff on energy and bro...,7,1,"['first', 'minister', 'john', 'swinney', 'said...",first minister john swinney said snp include s...,"['snp', 'include', 'social', 'tariff', 'energy...",snp include social tariff energy broadband gen...,0.9761,...,0.546628,0.157433,0.341095,0.658905,0.298832,0.701168,0.483395,0.516605,0.480627,0.519373
2,164,Reform UK insists its plans are “not just anot...,"Do Reform UK’s election claims on tax, immigra...",7,8,"['reform', 'insists', 'plan', 'another', 'part...",reform insists plan another party manifesto ex...,"['reform', 'uk', 'election', 'claim', 'tax', '...",reform uk election claim tax immigration envir...,0.9909,...,0.523378,0.000000,0.332418,0.667582,0.310975,0.689025,0.485103,0.514897,0.480939,0.519061
3,390,"“If you want politics as pantomime,” Sir Keir ...",Who is Jovan Owusu-Nepaul? Labour’s general el...,7,9,"['want', 'politics', 'pantomime', 'sir', 'keir...",want politics pantomime sir keir starmer said ...,"['jovan', 'owusunepaul', 'labour', 'general', ...",jovan owusunepaul labour general election cand...,0.9982,...,0.607156,0.025000,0.332650,0.667350,0.312563,0.687437,0.489046,0.510954,0.484018,0.515982
4,242,Shadow health secretary Wes Streeting has urge...,Streeting warns against complacency and giving...,6,4,"['shadow', 'health', 'secretary', 'wes', 'stre...",shadow health secretary wes streeting urged vo...,"['streeting', 'warns', 'complacency', 'giving'...",streeting warns complacency giving match back ...,0.9887,...,0.588329,0.125000,0.352597,0.647403,0.284916,0.715084,0.486872,0.513128,0.480360,0.519640


In [ ]:
article_table_s.columns

Index(['Article_id', 'Article', 'Title', 'Published_date_id', 'Source_id',
       'article_tokens_lem', 'article_tokens_lem_text', 'title_tokens_lem',
       'title_tokens_lem_text', 'article_sentiment_vader',
       'title_sentiment_vader', 'article_sentiment_textblob',
       'title_sentiment_textblob', 'article_sentiment_hybrid',
       'title_sentiment_hybrid', 'article_sentiment_bert_-',
       'article_sentiment_bert_+', 'title_sentiment_bert_-',
       'title_sentiment_bert_+', 'article_sentiment_roberta_-',
       'article_sentiment_roberta_+', 'title_sentiment_roberta_-',
       'title_sentiment_roberta_+'],
      dtype='object')

### Sentiment Labelling Process

In [ ]:
article_table_s['article_sentiment_bert_ro_+_avg'] = ((article_table_s['article_sentiment_bert_+'] + article_table_s['article_sentiment_roberta_+']) / 2)
article_table_s['article_sentiment_bert_ro_-_avg'] = ((article_table_s['article_sentiment_bert_-'] + article_table_s['article_sentiment_roberta_-']) / 2)
article_table_s['title_sentiment_bert_ro_+_avg'] = ((article_table_s['title_sentiment_bert_+'] + article_table_s['title_sentiment_roberta_+']) / 2)
article_table_s['title_sentiment_bert_ro_-_avg'] = ((article_table_s['title_sentiment_bert_-'] + article_table_s['title_sentiment_roberta_-']) / 2)

In [ ]:
article_table_s['article_bert_diff'] = abs(article_table_s['article_sentiment_bert_-'] - article_table_s['article_sentiment_bert_+'])
article_table_s['title_bert_diff'] = abs(article_table_s['title_sentiment_bert_-'] - article_table_s['title_sentiment_bert_+'])

article_table_s['article_roberta_diff'] = abs(article_table_s['article_sentiment_roberta_-'] - article_table_s['article_sentiment_roberta_+'])
article_table_s['title_roberta_diff'] = abs(article_table_s['title_sentiment_roberta_-'] - article_table_s['title_sentiment_roberta_+'])

article_table_s['article_avg_diff'] = abs(article_table_s['article_sentiment_bert_ro_-_avg'] - article_table_s['article_sentiment_bert_ro_+_avg'])
article_table_s['title_avg_diff'] = abs(article_table_s['title_sentiment_bert_ro_-_avg'] - article_table_s['title_sentiment_bert_ro_+_avg'])


# Define a function to label based on BERT sentiment differences
def label_bert_a(row):
    if row['article_bert_diff'] <= 0.3:
        return 'Neutral'
    elif row['article_sentiment_bert_+'] > row['article_sentiment_bert_-']:
        return 'Positive'
    elif row['article_sentiment_bert_-'] > row['article_sentiment_bert_+']:
        return 'Negative'
    return 'None'  # Default return to ensure the function always returns a value

# Define a function to label based on ROBERTA sentiment differences
def label_roberta_a(row):
    if row['article_roberta_diff'] <= 0.03:
        return 'Neutral'
    elif row['article_sentiment_roberta_+'] > row['article_sentiment_roberta_-']:
        return 'Positive'
    elif row['article_sentiment_roberta_-'] > row['article_sentiment_roberta_+']:
        return 'Negative'
    return 'None'  # Default return to ensure the function always returns a value

# Define a function to label based on ROBERTA sentiment differences
def label_avg_a(row):
    if row['article_avg_diff'] <= 0.2:
        return 'Neutral'
    elif row['article_sentiment_bert_ro_+_avg'] > row['article_sentiment_bert_ro_-_avg']:
        return 'Positive'
    elif row['article_sentiment_bert_ro_-_avg'] > row['article_sentiment_bert_ro_+_avg']:
        return 'Negative'
    return 'None'  # Default return to ensure the function always returns a value


# Apply the function to each row to create the new column 'Article_bert_label'
article_table_s['Article_bert_label'] = article_table_s.apply(label_bert_a, axis=1)
article_table_s['Article_roberta_label'] = article_table_s.apply(label_roberta_a, axis=1)
article_table_s['Article_avg_label'] = article_table_s.apply(label_avg_a, axis=1)

print(article_table_s['Article_bert_label'].value_counts())
print(article_table_s['Article_roberta_label'].value_counts())
print(article_table_s['Article_avg_label'].value_counts())

Article_bert_label
Positive    276
Neutral     130
Name: count, dtype: int64
Article_roberta_label
Neutral     290
Positive    116
Name: count, dtype: int64
Article_avg_label
Neutral     370
Positive     36
Name: count, dtype: int64


In [ ]:
# Define a function to label based on BERT sentiment differences
def label_bert_t(row):
    if row['title_bert_diff'] <= 0.3:
        return 'Neutral'
    elif row['title_sentiment_bert_+'] > row['title_sentiment_bert_-']:
        return 'Positive'
    elif row['title_sentiment_bert_-'] > row['title_sentiment_bert_+']:
        return 'Negative'
    return 'None'  # Default return to ensure the function always returns a value

# Define a function to label based on ROBERTA sentiment differences
def label_roberta_t(row):
    if row['title_roberta_diff'] <= 0.03:
        return 'Neutral'
    elif row['title_sentiment_roberta_+'] > row['title_sentiment_roberta_-']:
        return 'Positive'
    elif row['title_sentiment_roberta_-'] > row['title_sentiment_roberta_+']:
        return 'Negative'
    return 'None'  # Default return to ensure the function always returns a value

# Define a function to label based on ROBERTA sentiment differences
def label_avg_t(row):
    if row['title_avg_diff'] <= 0.2:
        return 'Neutral'
    elif row['title_sentiment_bert_ro_+_avg'] > row['title_sentiment_bert_ro_-_avg']:
        return 'Positive'
    elif row['title_sentiment_bert_ro_-_avg'] > row['title_sentiment_bert_ro_+_avg']:
        return 'Negative'
    return 'None'  # Default return to ensure the function always returns a value


# Apply the function to each row to create the new column 'title_bert_label'
article_table_s['title_bert_label'] = article_table_s.apply(label_bert_t, axis=1)
article_table_s['title_roberta_label'] = article_table_s.apply(label_roberta_t, axis=1)
article_table_s['title_avg_label'] = article_table_s.apply(label_avg_t, axis=1)

print(article_table_s['title_bert_label'].value_counts())
print(article_table_s['title_roberta_label'].value_counts())
print(article_table_s['title_avg_label'].value_counts())

title_bert_label
Positive    406
Name: count, dtype: int64
title_roberta_label
Positive    394
Neutral      12
Name: count, dtype: int64
title_avg_label
Positive    336
Neutral      70
Name: count, dtype: int64


In [ ]:
article_table_s.iloc[:5 , 27:]

,article_bert_diff,title_bert_diff,article_roberta_diff,title_roberta_diff,article_avg_diff,title_avg_diff,Article_bert_label,Article_roberta_label,Article_avg_label,title_bert_label,title_roberta_label,title_avg_label
0,0.353199,0.416110,0.028384,0.035424,0.190791,0.225767,Positive,Neutral,Neutral,Positive,Positive,Positive
1,0.317810,0.402336,0.033209,0.038746,0.175510,0.220541,Positive,Positive,Neutral,Positive,Positive,Positive
2,0.335163,0.378050,0.029794,0.038123,0.182478,0.208086,Positive,Neutral,Neutral,Positive,Positive,Positive
3,0.334701,0.374873,0.021909,0.031963,0.178305,0.203418,Positive,Neutral,Neutral,Positive,Positive,Positive
4,0.294805,0.430168,0.026256,0.039281,0.160531,0.234724,Neutral,Neutral,Neutral,Positive,Positive,Positive


### Quality / Performance checks

In [ ]:
import random

random_num_1 = random.randint(1, 299)
random_num_2 = random.randint(1, 299)

article_test = article_table_s[['Article', 'Article_avg_label', 'Article_bert_label', 'Article_roberta_label']]

if random_num_1 != random_num_2:
  article_test = article_test.iloc[random_num_1 : random_num_2, :].copy()

article_test.head()


,Article,Article_avg_label,Article_bert_label,Article_roberta_label
118,UK voters will head to the polls to elect the ...,Neutral,Positive,Neutral
119,"When Rishi Sunak, Britain’s Conservative prime...",Neutral,Positive,Neutral
120,Rishi Sunak has indicated that young people mi...,Neutral,Neutral,Neutral
121,Sir Keir Starmer has said that Jeremy Corbyn w...,Neutral,Neutral,Neutral
122,"From Rishi Sunak, we saw the good, the bad and...",Neutral,Positive,Neutral


In [ ]:
article_table_s['article_tokens_lem'][random_num_1]

"['voter', 'head', 'poll', 'elect', 'next', 'prime', 'minister', 'two', 'week', 'time', 'julypolitical', 'party', 'busy', 'campaigning', 'following', 'snap', 'election', 'announcement', 'rishi', 'sunak', 'revealed', 'vote', 'would', 'held', 'thursday', 'julythe', 'conservative', 'currently', 'trailing', 'labour', 'significant', 'margin', 'poll', 'latest', 'polling', 'figure', 'showing', 'labour', 'projected', 'potentially', 'secure', 'record', 'number', 'mp', 'sunak', 'danger', 'losing', 'seat', 'would', 'make', 'first', 'sitting', 'prime', 'minister', 'soother', 'highprofile', 'tory', 'mp', 'including', 'michael', 'gove', 'jeremy', 'hunt', 'jacob', 'reesmogg', 'liz', 'truss', 'also', 'facing', 'prospect', 'ousted', 'voterssome', 'tactical', 'voting', 'campaigner', 'eager', 'make', 'reality', 'conservative', 'sitting', 'party', 'government', 'likely', 'sharp', 'end', 'initiativesindeed', 'best', 'britain', 'campaign', 'group', 'say', 'want', 'inflict', 'heaviest', 'possible', 'election

In [ ]:
article_table_s.columns

Index(['Article_id', 'Article', 'Title', 'Published_date_id', 'Source_id',
       'article_tokens_lem', 'article_tokens_lem_text', 'title_tokens_lem',
       'title_tokens_lem_text', 'article_sentiment_vader',
       'title_sentiment_vader', 'article_sentiment_textblob',
       'title_sentiment_textblob', 'article_sentiment_hybrid',
       'title_sentiment_hybrid', 'article_sentiment_bert_-',
       'article_sentiment_bert_+', 'title_sentiment_bert_-',
       'title_sentiment_bert_+', 'article_sentiment_roberta_-',
       'article_sentiment_roberta_+', 'title_sentiment_roberta_-',
       'title_sentiment_roberta_+', 'article_sentiment_bert_ro_+_avg',
       'article_sentiment_bert_ro_-_avg', 'title_sentiment_bert_ro_+_avg',
       'title_sentiment_bert_ro_-_avg', 'article_bert_diff', 'title_bert_diff',
       'article_roberta_diff', 'title_roberta_diff', 'article_avg_diff',
       'title_avg_diff', 'Article_bert_label', 'Article_roberta_label',
       'Article_avg_label', 'title_bert_l

### Saving Final Article Sentiment Table

In [ ]:
article_table_s = article_table_s.drop(['article_sentiment_vader',
       'title_sentiment_vader', 'article_sentiment_textblob',
       'title_sentiment_textblob', 'article_sentiment_hybrid',
       'title_sentiment_hybrid', 'article_sentiment_bert_ro_+_avg',
       'article_sentiment_bert_ro_-_avg', 'title_sentiment_bert_ro_+_avg',
       'title_sentiment_bert_ro_-_avg','article_roberta_diff', 'title_roberta_diff', 'article_avg_diff',
       'title_avg_diff',
       'Article_avg_label',
       'title_avg_label'
        ], axis = 1)

In [ ]:
article_table_s.to_csv('article_table_s_f', index=False)